In [2]:
import numpy as np
import pandas as pd

# Load the dataset

In [535]:
#df = pd.read_csv('pred_hyper_label.csv')
df = pd.read_csv('Fake news meta/FakeNews_with_hyper_Binary_int(SVM)_1.csv')
#df = pd.read_csv('FakeNewsNet/mix.csv')
#df = pd.read_csv('pred_hyper.csv')


In [537]:
len(df)

2268

In [538]:
df.head()

,doc_id,title,mainText,veracity,mainText_preprocessed,upper_title,upper_mainText,punct_title,punct_mainText,exclamations_title,...,title_words_not_stopword,text_words_not_stopword,title_mainText,hyperpartisan,title_preprocessed,title_mainText_preprocessed,mainText_nouns,mainText_verbs,title_mainText_nouns,title_mainText_verbs
0,0,BREAKING STUDY PROVES TRUMP WAS RIGHT … EVERY ...,Results from a new study concluded that millio...,False,result new study conclude million could vote p...,10,0,1,39,1,...,13,204,BREAKING STUDY PROVES TRUMP WAS RIGHT … EVERY ...,1,breaking study prof trump wa right every citiz...,breaking study prof trump wa right every citiz...,result study million election president donald...,conclude vote suggest cost review announce rep...,BREAKING STUDY PROVES TRUMP WAS RIGHT … EVERY ...,BREAKING STUDY PROVES TRUMP WAS RIGHT … EVERY ...
1,1,President Trump Signed An Executive Order That...,Illegal immigrants do NOT have the same rights...,False,illegal immigrant right american citizen illeg...,0,3,1,17,0,...,12,108,President Trump Signed An Executive Order That...,1,president trump sign executive order prevent i...,president trump sign executive order prevent i...,immigrant right citizen immigrant welfare taxp...,come pay seem sign state enrol remove sign acc...,President Trump Signed An Executive Order That...,President Trump Signed An Executive Order That...
2,2,New Document Reveals Lynch Instructed Comey to...,"Ah, the infamous tarmac meeting. You know…the...",False,ah infamous tarmac meeting one former attorney...,0,1,1,16,0,...,9,77,New Document Reveals Lynch Instructed Comey to...,1,new document reveals lynch instructed comey hi...,new document reveals lynch instructed comey hi...,ah tarmac meeting attorney general loretta lyn...,meet suspect go get agree reveal tell do call ...,New Document Reveals Lynch Instructed Comey to...,New Document Reveals Lynch Instructed Comey to...
3,3,Newly Leaked Report Shows Hillary DID NOT Win ...,President Trump won the election fair and squa...,False,president trump win election fair square evide...,3,4,0,53,0,...,13,204,Newly Leaked Report Shows Hillary DID NOT Win ...,1,newly leak report show hillary win new dems pa...,newly leak report show hillary win new dems pa...,president trump election fair square evidence ...,pile suggest appear give report indicate regis...,Newly Leaked Report Shows Hillary DID NOT Win ...,Newly Leaked Report Shows Hillary DID NOT Win ...
4,4,KATIE COURIC SPEAKS OUT ON CANCELING CLINTON I...,Katie Couric has come out publicly and corrobo...,False,katie couric come publicly corroborate story f...,16,2,1,19,0,...,18,112,KATIE COURIC SPEAKS OUT ON CANCELING CLINTON I...,1,katie couric speaks canceling clinton intervie...,katie couric speaks canceling clinton intervie...,katie couric story sound tech hillary clinton ...,come corroborate hear refuse remain ask make r...,KATIE COURIC SPEAKS OUT ON CANCELING CLINTON I...,KATIE COURIC SPEAKS OUT ON CANCELING CLINTON I...


In [444]:
df['hyperpartisan'] = df['hyperpartisan'].astype(int)

# Merge features

In [539]:
# Merge hyperpartisan Label with text
features= [c for c in df.columns.values if c in ['title', 'hyperpartisan']]


X = df[features]
y = df['veracity']

In [507]:
features

['title', 'hyperpartisan']

In [508]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer, TfidfTransformer
from sklearn.preprocessing import StandardScaler, RobustScaler, MinMaxScaler
from sklearn.base import BaseEstimator, TransformerMixin
# Create selector class for text and numbers
class TextSelector(BaseEstimator, TransformerMixin):
    """Transformer to select a single column from the data frame to perform additional transformations on"""
    def __init__(self, key):
        self.key = key

    def fit(self, X, y=None):
        return self

    def transform(self, X):
        return X[self.key]
    
class NumberSelector(BaseEstimator, TransformerMixin):
    """For data grouped by feature, select subset of data at a provided key."""
    def __init__(self, key):
        self.key = key

    def fit(self, X, y=None):
        return self

    def transform(self, X):
        return X[[self.key]]

In [528]:
from sklearn.pipeline import Pipeline
# Building pipelines for each feature

# Test different scalers
scaler = StandardScaler()
#scaler = MinMaxScaler()
#scaler = RobustScaler()

text = Pipeline([
                ('selector', TextSelector(key='title')),
                ('vect', TfidfVectorizer(ngram_range=(1, 3))),                
            ])

hyperpartisan =  Pipeline([
                ('selector', NumberSelector(key='hyperpartisan')),
                ('standard', scaler),
            ])

In [529]:
# Feature Union
from sklearn.pipeline import FeatureUnion

feats = FeatureUnion([('title', text), 
                      ('hyperpartisan', hyperpartisan)])
                      

feature_processing = Pipeline([('feats', feats)])

In [530]:
from sklearn.svm import LinearSVC, SVC
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.svm import LinearSVC, SVC
from sklearn.linear_model import PassiveAggressiveClassifier
from sklearn.linear_model import LogisticRegression

pipeline = Pipeline([
        ('features', feats),
        ('classifier', SVC(C=1, kernel= 'linear', max_iter= 1000, tol=0.0001))
#        ('classifier', LogisticRegression(n_jobs=1, C=1e5))
    ])

# Grid Search

In [516]:
from sklearn.model_selection import GridSearchCV

hyperparameters = {#'features__title_mainText_verbs__vect__ngram_range': [(1,1), (1,2), (1,3)],
#                   'classifier__C': [1,10,100,1000],
#                   'classifier__max_iter': [100,500,1000]
                   'classifier__C': [0.01, 0.1, 1, 10, 100,1000,1e5,1e6],
                  }

clf = GridSearchCV(pipeline, hyperparameters, cv=10)
 
# Fit and tune model
clf.fit(X, y)

/anaconda3/lib/python3.6/site-packages/sklearn/preprocessing/data.py:625: DataConversionWarning: Data with input dtype int64 were all converted to float64 by StandardScaler.
  return self.partial_fit(X, y)
/anaconda3/lib/python3.6/site-packages/sklearn/base.py:465: DataConversionWarning: Data with input dtype int64 were all converted to float64 by StandardScaler.
  return self.fit(X, y, **fit_params).transform(X)
/anaconda3/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/anaconda3/lib/python3.6/site-packages/sklearn/pipeline.py:451: DataConversionWarning: Data with input dtype int64 were all converted to float64 by StandardScaler.
  Xt = transform.transform(Xt)
/anaconda3/lib/python3.6/site-packages/sklearn/pipeline.py:451: DataConversionWarning: Data with input dtype int64 were all converted to float64 by StandardScaler.
  Xt = transform.trans

/anaconda3/lib/python3.6/site-packages/sklearn/pipeline.py:451: DataConversionWarning: Data with input dtype int64 were all converted to float64 by StandardScaler.
  Xt = transform.transform(Xt)
/anaconda3/lib/python3.6/site-packages/sklearn/preprocessing/data.py:625: DataConversionWarning: Data with input dtype int64 were all converted to float64 by StandardScaler.
  return self.partial_fit(X, y)
/anaconda3/lib/python3.6/site-packages/sklearn/base.py:465: DataConversionWarning: Data with input dtype int64 were all converted to float64 by StandardScaler.
  return self.fit(X, y, **fit_params).transform(X)
/anaconda3/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/anaconda3/lib/python3.6/site-packages/sklearn/pipeline.py:451: DataConversionWarning: Data with input dtype int64 were all converted to float64 by StandardScaler.
  Xt = transform.trans

/anaconda3/lib/python3.6/site-packages/sklearn/preprocessing/data.py:625: DataConversionWarning: Data with input dtype int64 were all converted to float64 by StandardScaler.
  return self.partial_fit(X, y)
/anaconda3/lib/python3.6/site-packages/sklearn/base.py:465: DataConversionWarning: Data with input dtype int64 were all converted to float64 by StandardScaler.
  return self.fit(X, y, **fit_params).transform(X)
/anaconda3/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/anaconda3/lib/python3.6/site-packages/sklearn/pipeline.py:451: DataConversionWarning: Data with input dtype int64 were all converted to float64 by StandardScaler.
  Xt = transform.transform(Xt)
/anaconda3/lib/python3.6/site-packages/sklearn/pipeline.py:451: DataConversionWarning: Data with input dtype int64 were all converted to float64 by StandardScaler.
  Xt = transform.trans

/anaconda3/lib/python3.6/site-packages/sklearn/preprocessing/data.py:625: DataConversionWarning: Data with input dtype int64 were all converted to float64 by StandardScaler.
  return self.partial_fit(X, y)
/anaconda3/lib/python3.6/site-packages/sklearn/base.py:465: DataConversionWarning: Data with input dtype int64 were all converted to float64 by StandardScaler.
  return self.fit(X, y, **fit_params).transform(X)
/anaconda3/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/anaconda3/lib/python3.6/site-packages/sklearn/pipeline.py:451: DataConversionWarning: Data with input dtype int64 were all converted to float64 by StandardScaler.
  Xt = transform.transform(Xt)
/anaconda3/lib/python3.6/site-packages/sklearn/pipeline.py:451: DataConversionWarning: Data with input dtype int64 were all converted to float64 by StandardScaler.
  Xt = transform.trans

/anaconda3/lib/python3.6/site-packages/sklearn/preprocessing/data.py:625: DataConversionWarning: Data with input dtype int64 were all converted to float64 by StandardScaler.
  return self.partial_fit(X, y)
/anaconda3/lib/python3.6/site-packages/sklearn/base.py:465: DataConversionWarning: Data with input dtype int64 were all converted to float64 by StandardScaler.
  return self.fit(X, y, **fit_params).transform(X)
/anaconda3/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/anaconda3/lib/python3.6/site-packages/sklearn/pipeline.py:451: DataConversionWarning: Data with input dtype int64 were all converted to float64 by StandardScaler.
  Xt = transform.transform(Xt)
/anaconda3/lib/python3.6/site-packages/sklearn/pipeline.py:451: DataConversionWarning: Data with input dtype int64 were all converted to float64 by StandardScaler.
  Xt = transform.trans

...

/anaconda3/lib/python3.6/site-packages/sklearn/pipeline.py:451: DataConversionWarning: Data with input dtype int64 were all converted to float64 by StandardScaler.
  Xt = transform.transform(Xt)
/anaconda3/lib/python3.6/site-packages/sklearn/pipeline.py:451: DataConversionWarning: Data with input dtype int64 were all converted to float64 by StandardScaler.
  Xt = transform.transform(Xt)
/anaconda3/lib/python3.6/site-packages/sklearn/preprocessing/data.py:625: DataConversionWarning: Data with input dtype int64 were all converted to float64 by StandardScaler.
  return self.partial_fit(X, y)
/anaconda3/lib/python3.6/site-packages/sklearn/base.py:465: DataConversionWarning: Data with input dtype int64 were all converted to float64 by StandardScaler.
  return self.fit(X, y, **fit_params).transform(X)
/anaconda3/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  Future

/anaconda3/lib/python3.6/site-packages/sklearn/pipeline.py:451: DataConversionWarning: Data with input dtype int64 were all converted to float64 by StandardScaler.
  Xt = transform.transform(Xt)
/anaconda3/lib/python3.6/site-packages/sklearn/preprocessing/data.py:625: DataConversionWarning: Data with input dtype int64 were all converted to float64 by StandardScaler.
  return self.partial_fit(X, y)
/anaconda3/lib/python3.6/site-packages/sklearn/base.py:465: DataConversionWarning: Data with input dtype int64 were all converted to float64 by StandardScaler.
  return self.fit(X, y, **fit_params).transform(X)
/anaconda3/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/anaconda3/lib/python3.6/site-packages/sklearn/pipeline.py:451: DataConversionWarning: Data with input dtype int64 were all converted to float64 by StandardScaler.
  Xt = transform.trans

/anaconda3/lib/python3.6/site-packages/sklearn/preprocessing/data.py:625: DataConversionWarning: Data with input dtype int64 were all converted to float64 by StandardScaler.
  return self.partial_fit(X, y)
/anaconda3/lib/python3.6/site-packages/sklearn/base.py:465: DataConversionWarning: Data with input dtype int64 were all converted to float64 by StandardScaler.
  return self.fit(X, y, **fit_params).transform(X)
/anaconda3/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/anaconda3/lib/python3.6/site-packages/sklearn/pipeline.py:451: DataConversionWarning: Data with input dtype int64 were all converted to float64 by StandardScaler.
  Xt = transform.transform(Xt)
/anaconda3/lib/python3.6/site-packages/sklearn/pipeline.py:451: DataConversionWarning: Data with input dtype int64 were all converted to float64 by StandardScaler.
  Xt = transform.trans

/anaconda3/lib/python3.6/site-packages/sklearn/preprocessing/data.py:625: DataConversionWarning: Data with input dtype int64 were all converted to float64 by StandardScaler.
  return self.partial_fit(X, y)
/anaconda3/lib/python3.6/site-packages/sklearn/base.py:465: DataConversionWarning: Data with input dtype int64 were all converted to float64 by StandardScaler.
  return self.fit(X, y, **fit_params).transform(X)
/anaconda3/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/anaconda3/lib/python3.6/site-packages/sklearn/pipeline.py:451: DataConversionWarning: Data with input dtype int64 were all converted to float64 by StandardScaler.
  Xt = transform.transform(Xt)
/anaconda3/lib/python3.6/site-packages/sklearn/pipeline.py:451: DataConversionWarning: Data with input dtype int64 were all converted to float64 by StandardScaler.
  Xt = transform.trans

GridSearchCV(cv=10, error_score='raise-deprecating',
       estimator=Pipeline(memory=None,
     steps=[('features', FeatureUnion(n_jobs=None,
       transformer_list=[('title', Pipeline(memory=None,
     steps=[('selector', TextSelector(key='title')), ('vect', TfidfVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.float64'>, encoding='utf-8', input='c...penalty='l2', random_state=None, solver='warn',
          tol=0.0001, verbose=0, warm_start=False))]),
       fit_params=None, iid='warn', n_jobs=None,
       param_grid={'classifier__C': [0.01, 0.1, 1, 10, 100, 1000, 100000.0, 1000000.0]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring=None, verbose=0)

In [517]:
clf.best_params_

{'classifier__C': 100000.0}

In [417]:
clf.best_score_

0.7821869488536155

# Cross Validation

In [531]:
from sklearn.model_selection import cross_val_score
score = cross_val_score(pipeline, X, y, cv=10) 
score

/anaconda3/lib/python3.6/site-packages/sklearn/preprocessing/data.py:625: DataConversionWarning: Data with input dtype int64 were all converted to float64 by StandardScaler.
  return self.partial_fit(X, y)
/anaconda3/lib/python3.6/site-packages/sklearn/base.py:465: DataConversionWarning: Data with input dtype int64 were all converted to float64 by StandardScaler.
  return self.fit(X, y, **fit_params).transform(X)
/anaconda3/lib/python3.6/site-packages/sklearn/svm/base.py:244: ConvergenceWarning: Solver terminated early (max_iter=1000).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  % self.max_iter, ConvergenceWarning)
/anaconda3/lib/python3.6/site-packages/sklearn/pipeline.py:451: DataConversionWarning: Data with input dtype int64 were all converted to float64 by StandardScaler.
  Xt = transform.transform(Xt)
/anaconda3/lib/python3.6/site-packages/sklearn/preprocessing/data.py:625: DataConversionWarning: Data with input dtype int64 were all converted to float

array([0.73245614, 0.70484581, 0.71365639, 0.68722467, 0.83700441,
       0.59911894, 0.60792952, 0.74336283, 0.74778761, 0.73893805])

In [532]:
print (score.mean())

0.7112324371608919


In [533]:
from sklearn import metrics
scores = cross_val_score(pipeline, X, y, cv=10, scoring='f1_macro')
scores 

/anaconda3/lib/python3.6/site-packages/sklearn/preprocessing/data.py:625: DataConversionWarning: Data with input dtype int64 were all converted to float64 by StandardScaler.
  return self.partial_fit(X, y)
/anaconda3/lib/python3.6/site-packages/sklearn/base.py:465: DataConversionWarning: Data with input dtype int64 were all converted to float64 by StandardScaler.
  return self.fit(X, y, **fit_params).transform(X)
/anaconda3/lib/python3.6/site-packages/sklearn/svm/base.py:244: ConvergenceWarning: Solver terminated early (max_iter=1000).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  % self.max_iter, ConvergenceWarning)
/anaconda3/lib/python3.6/site-packages/sklearn/pipeline.py:451: DataConversionWarning: Data with input dtype int64 were all converted to float64 by StandardScaler.
  Xt = transform.transform(Xt)
/anaconda3/lib/python3.6/site-packages/sklearn/preprocessing/data.py:625: DataConversionWarning: Data with input dtype int64 were all converted to float

array([0.73240981, 0.70447877, 0.71256307, 0.68239235, 0.83695378,
       0.58756115, 0.60095992, 0.74090765, 0.74466314, 0.73778293])

In [534]:
print (scores.mean())

0.7080672582102601
